In [161]:
import pandas as pd
import numpy as np
import pyspark
import os
import shutil
import copy

In [162]:
#----------------------------------------------------------------
# GET RELATION ACTIVITIES CATEGORIES
#----------------------------------------------------------------
def getRelationActivitiesCategories(tuplesList, activitiesDict):
    size = len(tuplesList)
    relation = {}
    for i in range(0, size):
        activity = tuplesList[i][0]
        category = tuplesList[i][1]
        if category not in relation:
            relation[category] = [activitiesDict[activity]]
        else: 
            relation[category].append(activitiesDict[activity])
    return relation
#----------------------------------------------------------------
# GET HASH NAMES
#----------------------------------------------------------------
def gethHashedNames(namesList):
    names = {}
    cont = 0
    size = len(namesList)
    for i in range(0, size):
        if namesList[i] in names: continue
        names[namesList[i]] = cont
        cont += 1
    return names
#----------------------------------------------------------------
# HASH NAMES
#----------------------------------------------------------------
def hashNames(x, pos):
    names = {}
    cont = 0
    x = list(x)
    size = len(x)
    for i in range(0, size):
        if x[i][pos] in names:
            x[i][pos] = names[x[i][pos]]
            continue
        names[x[i][pos]] = cont
        cont += 1
        x[i][pos] = names[x[i][pos]]
    return x
#----------------------------------------------------------------
# TOOK TIMES TO SECONDS
#----------------------------------------------------------------
def TookTimesToMinutes(aTookTime):
    # '2018-04-21T16:22:49.000Z'
    aTookTime = aTookTime.split(aTookTime[10])[1]
    aTookTime = aTookTime.split(aTookTime[8])[0]
    #16:22:49
    aTookTime = aTookTime.split(':')
    aTookTime[0] = int(aTookTime[0])
    aTookTime[1] = int(aTookTime[1])
    aTookTime[2] = int(aTookTime[2])
    if (aTookTime[0] < 13):
        aTookTime[0] += 24
    aTookTime[0] = aTookTime[0]*60*60
    aTookTime[1] = aTookTime[1]*60
    aTookTime = aTookTime[0] + aTookTime[1] + aTookTime[2]
    return aTookTime
#----------------------------------------------------------------
# CHANGE TOOK TIMES
#----------------------------------------------------------------
def changeTookTimes(x):
    size = len(x[1])
    # name, [(w, t), (w, t), ....]
    for j in range(0, size): x[1][j][1] = TookTimesToMinutes(x[1][j][1])
    vec = []
    for j in range(0, size):
        if (j == 0): vec.append([x[1][j][0], 0])
        else:
            newTookTime = x[1][j][1] - x[1][0][1]
            # 14:00:00: 840 -> 0 -> vec[0]
            # 14:01:00: 900 -> 60 -> 
            # 14:02:00: 960 -> 120
            # 14:05:00:  -> 300
            vec.append([x[1][j][0], newTookTime])
    x[1] = vec
    return x
#----------------------------------------------------------------
# CHANGE FORMAT
#----------------------------------------------------------------
def changeFormat(x):
    size = len(x[1])
    for j in range(0, size): 
        x[1][j] = [x[0], x[1][j][0], x[1][j][1]]
    return x[1]
#----------------------------------------------------------------
# DELETE REPETITIONS
#----------------------------------------------------------------
def deleteRepetitions(x):
    repeated = {}
    indexToDelete = []
    size = len(x[1])
    for i in range(0, size):
        tookTime = x[1][i][1]
        if tookTime not in repeated:
            repeated[tookTime] = True
        else:
            indexToDelete.append(i)
    for i in indexToDelete: 
        x[1].pop(i)
    return x
#----------------------------------------------------------------
# SHOW ERRORS
#----------------------------------------------------------------
def showErrors(disney):
    disney = disney.map(lambda x: (x[2], (1, x[0])))
    disney = disney.reduceByKey(lambda x, y: (x[0]+y[0], x[1]))
    disney = disney.collect()
    return [x for x in disney if x[1][0] != 22]
#----------------------------------------------------------------
# REMOVE TOOK TIMES
#----------------------------------------------------------------
def removeTookTimes(disney, listTookTimes):
    return disney.filter(lambda x: x[2] not in listTookTimes)
#----------------------------------------------------------------
# GET TOOK TIMES MINUTES
#----------------------------------------------------------------
def getTookTimeMinutes(aTookTime):
    #'2018-04-21T13:14:57.000Z'
    aTookTime = aTookTime.split(aTookTime[10])[1]
    aTookTime = aTookTime.split(aTookTime[8])[0]
    #13:14:57
    aTookTime = aTookTime.split(':')
    hours = int(aTookTime[0])
    if (hours < 13):
        hours += 24
    minutes = int(aTookTime[1])
    return hours*60 + minutes
#----------------------------------------------------------------
# REMOVE TOOK TIMES EVERY GAP
#----------------------------------------------------------------
def removeTookTimesEveryGap(reg, gap):
    vec = []
    size = len(reg[1])
    vec.append(reg[1][0])
    T0 = getTookTimeMinutes(reg[1][0][1])
    for i in range(1, size):
        T1 = getTookTimeMinutes(reg[1][i][1])
        if (T1 - T0 >= gap):
            T0 = T1
            vec.append(reg[1][i])
    reg[1] = vec
    return reg
#----------------------------------------------------------------
# SHOW TOOK TIME HASH
#----------------------------------------------------------------
def showTookTimehash(tookTimes):
    size = len(tookTimes)
    for i in range(0, size):
        print i, ": ", tookTimes[i][1]
#----------------------------------------------------------------
# GET DICT TOOK TIMES
#----------------------------------------------------------------
def getDictTookTimes(tookTimes, disney, MAX_TOOKTIME):
    tookTimesInSeconds = sc.parallelize(disney).map(lambda x: x[2]).collect()[0:MAX_TOOKTIME]
    dicc = {}
    for i in range(0, MAX_TOOKTIME):
        aTookTime = tookTimes[i][1]
        aTookTime = aTookTime.split(aTookTime[10])[1]
        aTookTime = aTookTime.split(aTookTime[8])[0]
        dicc[tookTimesInSeconds[i]] = aTookTime
    return dicc

In [163]:
gap = 45
carpet = "modelo" + str(gap)
dataName = carpet + "/datos" + str(gap) + ".txt"
modelName = carpet + "/model" + str(gap) + ".txt"

#  Parse Disney

In [164]:
disneyRoot = "SetDeDatos/disney.csv"
disney = spark.read.load(disneyRoot, format="csv", header=True).rdd
disney = disney.map(lambda line: list([str(x) for x in line]))
disney = disney.map(lambda x: [x[1], int(x[2]), x[3]])
disney = disney.map(lambda x: [x[0], [x[1], x[2]]]).groupByKey()
disney = disney.map(lambda x: [x[0], list(x[1])])
disney = disney.map(lambda x: removeTookTimesEveryGap(x, gap))
tookTimes = disney.collect()[0][1]
disney = disney.map(lambda x: deleteRepetitions(x))
disney = disney.map(lambda x: changeTookTimes(x))
disney = disney.mapPartitions(lambda x: hashNames(x, 0))
disney = disney.flatMap(lambda x: changeFormat(x))
disney = disney.map(lambda x: [x[0], x[1]*60, x[2]])
print "Errors: ", showErrors(disney)
print "Tooktimes: "
print showTookTimehash(tookTimes)
print "Parsed Disney: "
print disney.take(5)
disney = disney.collect()
tookTimeDicc = getDictTookTimes(tookTimes, disney, len(tookTimes))

Errors:  []
Tooktimes: 
0 :  2018-04-21T13:14:57.000Z
1 :  2018-04-21T13:59:48.000Z
2 :  2018-04-21T14:44:48.000Z
3 :  2018-04-21T15:29:48.000Z
4 :  2018-04-21T16:14:49.000Z
5 :  2018-04-21T16:59:49.000Z
6 :  2018-04-21T17:44:49.000Z
7 :  2018-04-21T18:29:49.000Z
8 :  2018-04-21T19:14:49.000Z
9 :  2018-04-21T19:59:49.000Z
10 :  2018-04-21T20:44:50.000Z
11 :  2018-04-21T21:29:50.000Z
12 :  2018-04-21T22:14:50.000Z
13 :  2018-04-21T22:59:50.000Z
14 :  2018-04-21T23:44:50.000Z
15 :  2018-04-22T00:29:50.000Z
16 :  2018-04-22T01:14:51.000Z
None
Parsed Disney: 
[[0, 600, 0], [0, 600, 2691], [0, 900, 5391], [0, 1200, 8091], [0, 2100, 10792]]


#  Parse Description

In [165]:
descriptionRoot = "SetDeDatos/descripcion.csv"
description = spark.read.load(descriptionRoot, format="csv", header=True).rdd
description = description.map(lambda line: list([str(x) for x in line]))
#-----------------------------------------------------------------------------
categoriesDict = gethHashedNames(description.map(lambda x: x[1]).collect())
activitiesDict = gethHashedNames(description.map(lambda x: x[0]).collect())
tupleList = description.map(lambda x: (x[0], x[1])).collect()
relation = getRelationActivitiesCategories(tupleList, activitiesDict)
#-----------------------------------------------------------------------------
categoriesDict["ACCION"] = categoriesDict.pop("accion")
categoriesDict["BOTES"] = categoriesDict.pop("botes")
categoriesDict["FUTURO"] = categoriesDict.pop("futuro")
categoriesDict["MONTANA"] = categoriesDict.pop("monta??a")
categoriesDict["MUSICAL"] = categoriesDict.pop("musical")
categoriesDict["NINOS"] = categoriesDict.pop("ni??os")

relation["ACCION"] = relation.pop("accion")
relation["BOTES"] = relation.pop("botes")
relation["FUTURO"] = relation.pop("futuro")
relation["MONTANA"] = relation.pop("monta??a")
relation["MUSICAL"] = relation.pop("musical")
relation["NINOS"] = relation.pop("ni??os")
#-----------------------------------------------------------------------------
print "Categorias: ", categoriesDict
print " "
print "Actividades: ", activitiesDict
print " "
#-----------------------------------------------------------------------------
description = description.mapPartitions(lambda x: hashNames(x, 0))
description = description.mapPartitions(lambda x: hashNames(x, 1))
description = description.map(lambda x: [x[0], x[1], int(x[2]), int(x[3])])
print "Description: "
description.collect()
#-----------------------------------------------------------------------------

Categorias:  {'ACCION': 4, 'BOTES': 5, 'MONTANA': 2, 'NINOS': 3, 'FUTURO': 1, 'MUSICAL': 0}
 
Actividades:  {'"""its a small world"""': 18, 'Enchanted Tales with Belle': 0, 'Astro Orbiter': 1, 'Haunted Mansion': 2, 'Space Mountain': 3, 'Seven Dwarfs Mine Train ': 4, 'Under the Sea - Journey of The Little Mermaid': 5, 'The Barnstormer': 6, 'Monsters, Inc. Laugh Floor': 7, 'Big Thunder Mountain Railroad': 8, 'The Many Adventures of Winnie the Pooh': 9, 'Splash Mountain': 10, 'The Magic Carpets of Aladdin': 11, 'Prince Charming Regal Carrousel': 12, 'Buzz Lightyears Space Ranger Spin': 13, 'Tomorrowland Speedway': 14, 'Mickeys PhilharMagic': 15, 'Mad Tea Party': 21, 'Pirates of the Caribbean': 17, 'Dumbo the Flying Elephant': 19, 'Peter Pans Flight': 20, 'Jungle Cruise': 16}
 
Description: 


[[0, 0, 3, 0],
 [1, 1, 4, 0],
 [2, 1, 5, 1],
 [3, 2, 10, 1],
 [4, 2, 7, 1],
 [5, 3, 2, 1],
 [6, 3, 3, 0],
 [7, 3, 4, 0],
 [8, 2, 6, 1],
 [9, 3, 1, 0],
 [10, 2, 9, 0],
 [11, 3, 2, 0],
 [12, 3, 0, 0],
 [13, 4, 6, 1],
 [14, 1, 2, 1],
 [15, 0, 4, 0],
 [16, 5, 2, 0],
 [17, 5, 6, 1],
 [18, 3, 1, 1],
 [19, 3, 0, 0],
 [20, 3, 1, 0],
 [21, 3, 2, 1]]

# Agrego la comida

In [166]:
def copyList(aList, size):
    vec = copy.copy(aList)
    for i in range(0, size):
        vec[i] = copy.copy(aList[i])
    return vec
def DescriptionToDict(description):
    dicc = {}
    for row in description:
        dicc[row[0]] = row[1:len(row)]
    return dicc
def DisneyToDict(disney):
    disney = sc.parallelize(disney)
    disney = disney.map(lambda x: [x[0], [x[1], x[2], x[3], x[4], x[5]]])
    disney = disney.groupByKey().map(lambda x: [x[0], list(x[1])])
    disney = disney.collect()
    dicc = {}
    for row in disney:
        dicc[row[0]] = row[1:len(row)][0]
    return dicc

In [167]:
waitTimeFood = 40*60
SizeTookTimes = len(tookTimes)
size = len(activitiesDict.keys())
activitiesDict["food"] = size
categoriesDict["food"] = len(categoriesDict.keys())
vec = copyList(disney[0:SizeTookTimes], SizeTookTimes)
for i in range(0, SizeTookTimes):
    vec[i][0] = activitiesDict["food"]
    vec[i][1] = waitTimeFood
disney = disney+vec
description = description.collect()
description.append([activitiesDict["food"], categoriesDict["food"], 10, 1])
descriptionDict = DescriptionToDict(description)
description = sc.parallelize(description)
sizeDisney = len(disney)
for i in range(0, sizeDisney):
    activity = disney[i][0]
    disney[i].append(descriptionDict[activity][0])
    disney[i].append(descriptionDict[activity][1])
    disney[i].append(descriptionDict[activity][2])
disneyDict = DisneyToDict(disney)

In [168]:
MAX_ACTIVITIES = len(activitiesDict.values())
MAX_IMPRESCINDIBLES = len(description.map(lambda x: [x[3], 1]).filter(lambda x: x[0] == 1).collect())
MAX_TOOKTIME = len(tookTimes)

In [169]:
description = description.collect()

In [170]:
def sortByPos(disney, pos):
    return disney.sortBy(lambda x: -x[pos]).collect()
def sortByPosList(rdd, posList, FL):
    allPos = range(0, len(rdd.collect()[0]))
    rdd = rdd.map(lambda x: [[x[i] for i in posList], [x[i] for i in allPos if i not in posList]])
    rdd = rdd.sortByKey(False)
    rdd = rdd.map(lambda x: x[1]+x[0]).map(lambda x: [x[i] for i in FL])
    return rdd.collect()

In [171]:
disney = sortByPosList(sc.parallelize(disney), [5, 4, 2], [0, 1, 5, 2, 4, 3])
description = sortByPosList(sc.parallelize(description), [3, 2], [0, 1, 3, 2])

# Heuristica

In [172]:
import time

In [183]:
def thereIsOverlap(activityInTookTime, madeActivities, posDisneyDicc):
    size = len(madeActivities)
    for i in range(0, size):
        I_a = activityInTookTime[posDisneyDicc["tooktime"]]
        W_a = activityInTookTime[posDisneyDicc["waittime"]]
        F_a = I_a + W_a
        I_b = madeActivities[i][posDisneyDicc["tooktime"]]
        if F_a > I_b and I_a <= I_b:
            return True
    return False
def isEmpty(disneyDict):
    for key in disneyDict.keys():
        if (disneyDict[key]):
            return False
    return True
def getKeyfromValue(dicc, aValue):
    for key, value in dicc.iteritems():
        if value == aValue:
            return key
    return None
def copyDicc(dicc):
    diccCopy = {}
    for key in dicc.keys():
        diccCopy[key] = copy.copy(dicc[key])
    return diccCopy

In [184]:
posDisneyDicc = {
    "activity": 0,
    "waittime": 1,
    "tooktime": 2,
    "category": 3,
    "points": 4,
    "imprescindible": 5
}

In [204]:
imprescindibles = 0
activitiesQuant = {}
madeActivities = []
for i in range(0, MAX_ACTIVITIES):
    activitiesQuant[activitiesDict.values()[i]] = 0

activitiesList = copy.copy(description)
iteration = 0
activityPosition = 0
start = time.time()
disneyDictCopy = copyDicc(disneyDict)
while (imprescindibles < MAX_IMPRESCINDIBLES or not isEmpty(disneyDictCopy)):
    iteration += 1
    if activityPosition >= len(activitiesList):
        activityPosition = 0
    activity = activitiesList[activityPosition][0]
    activityInfo = disneyDictCopy[activity]
    ActivityQuantityInDifferentTookTimes = len(activityInfo)
    i = len(activityInfo)
    while activityInfo:
        if i >= len(activityInfo):
            i = 0
        AnActivityInATookTime = [activity] + activityInfo[i]
        isImprescindible = AnActivityInATookTime[posDisneyDicc["imprescindible"]]
        IM = isImprescindible
        if (imprescindibles < MAX_IMPRESCINDIBLES and not isImprescindible):
            break
        aTookTime = tookTimeDicc[AnActivityInATookTime[posDisneyDicc["tooktime"]]]
        waitime = AnActivityInATookTime[posDisneyDicc["waittime"]]/60
        if not thereIsOverlap(AnActivityInATookTime, madeActivities, posDisneyDicc):
            #print "Not overlap: ", activity, "in tooktime: ", aTookTime, "and waittime: ", waitime, "IM: ", IM
            activityInfo.pop(i)
            madeActivities.append(AnActivityInATookTime)
            activitiesQuant[activity] += 1
            if isImprescindible and activitiesQuant[activity] == 1:
                imprescindibles += 1
            break
        else:
            #print "overlap: ", activity, "in tooktime: ", aTookTime, "and waittime: ", waitime, "IM: ", IM
            activityInfo.pop(i)
    activityPosition += 1
madeActivities = sortByPos(sc.parallelize(madeActivities), 2)
end = time.time()
totalTime = end - start
print "Time used: " + str(totalTime) + " seconds"
print "iteration: ", iteration, " - Size Disney: ", len(disney)
print "Cantidad de imprescindibles", imprescindibles, "de un total de ", MAX_IMPRESCINDIBLES 

Time used: 0.365009069443 seconds
iteration:  40  - Size Disney:  391
Cantidad de imprescindibles 11 de un total de  11


In [205]:
size = len(madeActivities)
for i in range(0, size):
    tookTime = madeActivities[i][posDisneyDicc["tooktime"]]
    madeActivities[i][posDisneyDicc["tooktime"]] = tookTimeDicc[tookTime]
    madeActivities[i][posDisneyDicc["waittime"]] = madeActivities[i][posDisneyDicc["waittime"]]/60
    activity = madeActivities[i][posDisneyDicc["activity"]]
    madeActivities[i][posDisneyDicc["activity"]] = getKeyfromValue(activitiesDict, activity)
madeActivities

[['The Magic Carpets of Aladdin', 5, '01:14:51', 3, 2, 0],
 ['The Barnstormer', 25, '00:29:50', 3, 3, 0],
 ['Enchanted Tales with Belle', 25, '23:44:50', 0, 3, 0],
 ['Mickeys PhilharMagic', 30, '22:59:50', 0, 4, 0],
 ['Monsters, Inc. Laugh Floor', 40, '22:14:50', 3, 4, 0],
 ['Astro Orbiter', 20, '21:29:50', 1, 4, 0],
 ['Splash Mountain', 40, '20:44:50', 2, 9, 0],
 ['"""its a small world"""', 45, '19:59:49', 3, 1, 1],
 ['Mad Tea Party', 20, '19:14:49', 3, 2, 1],
 ['Tomorrowland Speedway', 50, '18:29:49', 1, 2, 1],
 ['Under the Sea - Journey of The Little Mermaid', 125, '17:44:49', 3, 2, 1],
 ['Haunted Mansion', 40, '16:59:49', 1, 5, 1],
 ['Pirates of the Caribbean', 50, '16:14:49', 5, 6, 1],
 ['Big Thunder Mountain Railroad', 20, '15:29:48', 2, 6, 1],
 ['Seven Dwarfs Mine Train ', 75, '14:44:48', 2, 7, 1],
 ['food', 40, '13:59:48', 6, 10, 1],
 ['Space Mountain', 10, '13:14:57', 2, 10, 1],
 ['Buzz Lightyears Space Ranger Spin', 0, '13:14:57', 4, 6, 1]]

In [ ]:
size = len(activitiesQuant)
act = {}
for key in activitiesQuant.keys():
    madeActivities[i][posDisneyDicc["tooktime"]] = tookTimeDicc[tookTime]
    madeActivities[i][posDisneyDicc["waittime"]] = madeActivities[i][posDisneyDicc["waittime"]]/60
    activity = madeActivities[i][posDisneyDicc["activity"]]
    madeActivities[i][posDisneyDicc["activity"]] = getKeyfromValue(activitiesDict, activity)
madeActivities
print "Cuantas veces se hizo cada actividad: "

# Puntaje Final 

In [ ]:
size = len(madeActivities)
finalPoints = 0
for i in range(0, size):
    finalPoints += madeActivities[i][posDisneyDicc["points"]]
print "finalPoints", finalPoints